# Run analysis on multiple files

In [8]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

In [17]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/FinerResolution_BendingStiffness'
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-06-23'
data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults'
# data_folder = '/home/deepak/Dropbox/LacryModeling/ModellingResults/StochasticActivity/NormalDistribution'


print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.'):
            files_list.append(os.path.join(dirs,fileNames))
#         if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
#             files_list.append(os.path.join(dirs,fileNames))

print('Simulation files: ', files_list)

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)

    # Create a data-packet to save for further analysis
    # Collect simulation metadata
#     metadata_path = os.path.join(root_folder, 'metadata.csv')
#     assert(os.path.exists(metadata_path))
#     df = pd.read_csv(metadata_path)
    
    # @@@ Better to use Metadata from the HDF5 file so there is no possible delinking of data and metadata
    df = pd.DataFrame({'N particles':[filament.Np],'radius':[filament.radius],'bond length':[filament.b0],
                       'spring constant':[filament.k], 'kappa_hat':[filament.kappa_hat], 
                       'force strength':[filament.F0], 'stresslet strength':[filament.S0], 
                       'potDipole strength':[filament.D0], 'simulation type':[filament.sim_type], 
                       'boundary condition 0':[filament.bc[0]], 'boundary condition 1':[filament.bc[-1]], 
                       ' activity time scale':[filament.activity_timescale], 'viscosity':[filament.mu]})
    
    
    periodic_flag = None 
    min_period = None
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(df[' activity time scale']*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period = filament.classify_filament_dynamics()
    else:
        simulation_completed = False

    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['period'] = min_period
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    
    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))




['2021-08-16', '2021-08-15_new']
Simulation files:  ['/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-16/SimResults_Np_32_Shape_line_kappa_hat_6_k_24.12_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point/SimResults_05.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-16/SimResults_Np_32_Shape_line_kappa_hat_6_k_24.12_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point/SimResults_03.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-16/SimResults_Np_32_Shape_line_kappa_hat_6_k_24.12_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point/SimResults_02.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-16/SimResults_Np_32_Shape_line_kappa_hat_6_k_24.12_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_point/SimResults_00.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/2021-08-16/SimResults_Np_32_Shape_line_kappa_hat_6_k_24.12_b0_2.1_F_0_S_0_D_1.5_activityTime_750_simType_poin

In [18]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


100%|██████████| 57/57 [00:31<00:00,  1.84it/s]
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   31.2s
[Parallel(n_jobs=12)]: Done  40 out of  57 | elapsed:   45.4s remaining:   19.3s
[Parallel(n_jobs=12)]: Done  46 out of  57 | elapsed:   47.6s remaining:   11.4s
[Parallel(n_jobs=12)]: Done  52 out of  57 | elapsed:   53.8s remaining:    5.2s
[Parallel(n_jobs=12)]: Done  57 out of  57 | elapsed:   55.5s finished
